# Czech Wikipedia in the covid era: Pageviews

This notebook shows data about pageviews for Czech Wikipedia pages during years 2020 and 2019.

Created by Martin Urbanec, Wikimedia Czech Republic. WIP notebook, do not rely on (yet).

In [1]:
from wmfdata import spark
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Configuration

In [2]:
SNAPSHOT = '2021-05'
PROJECT='cs.wikipedia'
DBNAME = 'cswiki'

spark.get_session(type='yarn-large');

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


## Helper methods

In [3]:
def merge_dataframes(file_2019, file_2020, suffixes=('_2019', '_2020')):
    df_2019 = pd.read_csv(file_2019, sep='\t')
    df_2019['date'] = df_2019.date.str.replace('2019-', 'year-')
    df_2019.set_index('date', inplace=True)

    df_2020 = pd.read_csv(file_2020, sep='\t')
    df_2020['date'] = df_2020.date.str.replace('2020-', 'year-')
    df_2020.set_index('date', inplace=True)

    df = df_2019.merge(df_2020, left_index=True, right_index=True, suffixes=suffixes)
    df.reset_index(inplace=True)
    df['date'] = df.date.str.replace('year-', '2020-')
    df.set_index('date', inplace=True)
    
    return df

In [4]:
def merge_multiple_dataframes(dfs, main_year='2020', values='views'):
    formattedDfs = []
    for df in dfs:
        year = df.date[0].split('-')[0]
        df['year'] = [year] * len(df.index)
        df['date'] = df.date.str.replace('%s-' % year, '%s-' % main_year)
        formattedDfs.append(df)
    
    res = pd.concat(formattedDfs)
    return res.pivot_table(index='date', columns=['year'], values=values, fill_value=0, aggfunc=sum)

## Total pageviews at Czech Wikipedia
Daily pageviews of all Czech Wikipedia pages during 2020 and 2019.

In [5]:
def get_views_per_year(year):
    return spark.run('''
    SELECT
        CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS `date`,
        SUM(view_count) AS views
    FROM wmf.projectview_hourly
    WHERE
            year={year}
        AND project="{project}"
        
        -- only user traffic
        AND agent_type='user'
    GROUP BY `date`
    ORDER BY `date`
    '''.format(
        project=PROJECT,
        year=year
    ))

In [6]:
df = get_views_per_year(2020)
df.to_csv('data/cswiki_pageviews_2020.tsv', sep='\t', index=False)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [7]:
df = get_views_per_year(2019)
df.to_csv('data/cswiki_pageviews_2019.tsv', sep='\t', index=False)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [8]:
df = get_views_per_year(2018)
df.to_csv('data/cswiki_pageviews_2018.tsv', sep='\t', index=False)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [9]:
df = get_views_per_year(2017)
df.to_csv('data/cswiki_pageviews_2017.tsv', sep='\t', index=False)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [10]:
df = merge_multiple_dataframes([pd.read_csv(x, sep='\t') for x in [
    'data/cswiki_pageviews_2017.tsv',
    'data/cswiki_pageviews_2018.tsv',
    'data/cswiki_pageviews_2019.tsv',
    'data/cswiki_pageviews_2020.tsv'
]])
df.to_csv('data/cswiki_pageviews_2017_2020.tsv', sep='\t')
df.head()

year,2017,2018,2019,2020
date,,,,
2020-01-01,2285584,3062661,2437416,2725376
2020-01-02,2831320,3409387,2584507,2587558
2020-01-03,2807273,2959980,2483793,2501301
2020-01-04,2863276,2720977,2339206,2783929
2020-01-05,3258482,2508048,2686137,3111024


## Pageviews difference -- helper methods

This section has helper methods to calculate pageview differences below.

In [11]:
def calculate_pageviews_difference(month, day):
    fillPageviewsQueryTemplate = '''
    INSERT INTO urbanecm.blogpost_covid_and_cswiki_pageviews

    SELECT
        year,
        month,
        day,
        page_title,
        SUM(view_count) AS views_{year}
    FROM wmf.pageview_hourly
    WHERE
            year = {year}
        AND month = {month}
        AND day = {day}
        AND project = '{project}'
        
        -- user traffic only
        AND agent_type='user'
        
        -- articles only
        AND namespace_id=0
        
        -- exclude false positive
        AND page_title != 'Venuše_(planeta)'
        AND page_title != '-'
    GROUP BY
        year,
        month,
        day,
        page_title
    '''
    
    queries = [
        'DROP TABLE IF EXISTS urbanecm.blogpost_covid_and_cswiki_pageviews',
        '''
        CREATE TABLE urbanecm.blogpost_covid_and_cswiki_pageviews(
            year bigint,
            month bigint,
            day bigint,
            page_title string,
            views bigint
        )
        ''',
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2019, month=month, day=day),
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2020, month=month, day=day),
        '''
        SELECT
            year,
            month,
            day,
            SUM(views)
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        GROUP BY
            year,
            month,
            day
        '''
    ]

    # fill temp pageviews table -- urbanecm.blogpost_covid_and_cswiki_pageviews
    spark.run(queries)
    
    # return data
    return spark.run('''
    WITH pv_2020 AS (
        SELECT
            month, day, page_title, views AS views_2020
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2020
    ),
    pv_2019 AS (
        SELECT
            month, day, page_title, views AS views_2019
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2019
    )

    SELECT
        pv_2020.month,
        pv_2020.day,
        pv_2020.page_title,
        views_2020,
        views_2019,
        views_2020 - views_2019 AS views_difference
    FROM pv_2020
    JOIN pv_2019 ON
        pv_2020.month=pv_2019.month
        AND pv_2020.day=pv_2019.day
        AND pv_2020.page_title=pv_2019.page_title
    ORDER BY views_difference DESC
    LIMIT 20
    ''')

In [12]:
def calculate_pageviews_difference_month(month):
    fillPageviewsQueryTemplate = '''
    INSERT INTO urbanecm.blogpost_covid_and_cswiki_pageviews

    SELECT
        year,
        month,
        page_title,
        SUM(view_count) AS views_{year}
    FROM wmf.pageview_hourly
    WHERE
            year = {year}
        AND month = {month}
        AND project = '{project}'
        
        -- user traffic only
        AND agent_type='user'
        
        -- articles only
        AND namespace_id=0
        
        -- exclude false positive articles
        AND page_title != 'Venuše_(planeta)'
        AND page_title != '-'
    GROUP BY
        year,
        month,
        page_title
    '''
    
    queries = [
        'DROP TABLE IF EXISTS urbanecm.blogpost_covid_and_cswiki_pageviews',
        '''
        CREATE TABLE urbanecm.blogpost_covid_and_cswiki_pageviews(
            year bigint,
            month bigint,
            page_title string,
            views bigint
        )
        ''',
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2019, month=month),
        fillPageviewsQueryTemplate.format(project=PROJECT, year=2020, month=month),
        '''
        SELECT
            year,
            month,
            SUM(views)
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        GROUP BY
            year,
            month
        '''
    ]

    # fill temp pageviews table -- urbanecm.blogpost_covid_and_cswiki_pageviews
    spark.run(queries, session_type="yarn-large")
    
    # return data
    return spark.run('''
    WITH pv_2020 AS (
        SELECT
            month, page_title, views AS views_2020
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2020
    ),
    pv_2019 AS (
        SELECT
            month, page_title, views AS views_2019
        FROM urbanecm.blogpost_covid_and_cswiki_pageviews
        WHERE
            year=2019
    )

    SELECT
        pv_2020.month,
        pv_2020.page_title,
        views_2020,
        COALESCE(views_2019, 0L) AS views_2019,
        views_2020 - COALESCE(views_2019, 0L) AS views_difference
    FROM pv_2020
    LEFT JOIN pv_2019 ON
        pv_2020.month=pv_2019.month
        AND pv_2020.page_title=pv_2019.page_title
    ORDER BY views_difference DESC
    LIMIT 20
    ''')

## Pageviews -- biggest difference -- total

In this section, I at months with higher projectviews in 2020 than in 2019 in more details, to figure out if the pandemic is the cause of the increased page views. For each of the months, I calculate pageviews of all articles in 2020 and 2019 (for that month only) and show 20 with the top difference.

An alternative could be to use [Topviews](https://pageviews.toolforge.org/topviews), but that tool looks at top visited articles in any given month, without any regard to the previous year(s).

### March

In [13]:
df = calculate_pageviews_difference_month(month=3)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_March.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,3,Pandemie_COVID-19_v_Česku,459649,0,459649
1,3,Pandemie_COVID-19,430442,0,430442
2,3,Koronavirus,340236,239,339997
3,3,Španělská_chřipka,234138,1676,232462
4,3,Epidemie_koronaviru_SARS-CoV-2,204541,0,204541
5,3,COVID-19,198428,0,198428
6,3,Epidemie_koronaviru_SARS-CoV-2_v_Česku,190229,0,190229
7,3,SARS-CoV-2,168047,0,168047
8,3,Morfologie_květu,162602,1289,161313
9,3,Adam_Vojtěch,161560,3160,158400


### April

In [14]:
df = calculate_pageviews_difference_month(month=4)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_April.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,4,Emanuel_Moravec,132881,2169,130712
1,4,Pandemie_COVID-19_v_Česku,130493,0,130493
2,4,Pandemie_covidu-19_v_Česku,91692,0,91692
3,4,Seznam_dílů_seriálu_Slunečná,77946,0,77946
4,4,Adam_Vojtěch,71685,1668,70017
5,4,Pandemie_COVID-19,69201,0,69201
6,4,Španělská_chřipka,63788,1041,62747
7,4,Koronavirus,52906,217,52689
8,4,Pandemie_covidu-19,49218,0,49218
9,4,Slunečná_(seriál),47855,0,47855


### July

In [15]:
df = calculate_pageviews_difference_month(month=7)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_July.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,7,Kateřina_II._Veliká,100535,2117,98418
1,7,Pandemie_covidu-19_v_Chorvatsku,33257,0,33257
2,7,Mistrovství_Evropy_ve_fotbale_2004,34233,1681,32552
3,7,Miloš_Jakeš,39798,9926,29872
4,7,Jiří_Procházka_(sportovec),28264,1267,26997
5,7,Ludvík_XIV.,29809,3448,26361
6,7,Petr_III._Ruský,26858,696,26162
7,7,Dýmějový_mor,27269,1668,25601
8,7,Jan_Skopeček,26324,2498,23826
9,7,Mistrovství_Evropy_ve_fotbale_2000,24165,491,23674


### October

In [16]:
df = calculate_pageviews_difference_month(month=10)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_October.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,10,Roman_Prymula,115742,303,115439
1,10,Covid-19,81618,0,81618
2,10,Jan_Blatný,72971,0,72971
3,10,Alois_Rašín,57214,3913,53301
4,10,Tvoje_tvář_má_známý_hlas_(7._řada),53200,156,53044
5,10,Španělská_chřipka,51296,2224,49072
6,10,Pandemie_covidu-19_v_Česku,45898,0,45898
7,10,Náhorní_Karabach,45211,1359,43852
8,10,Jaroslav_Flegr,44367,671,43696
9,10,Martin_Havelka,44094,919,43175


### November

In [17]:
df = calculate_pageviews_difference_month(month=11)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_November.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,11,Joe_Biden,182100,1026,181074
1,11,Donald_Trump,103132,6932,96200
2,11,Volby_prezidenta_USA_2020,84993,1420,83573
3,11,Volba_prezidenta_Spojených_států_amerických,84117,1033,83084
4,11,Diego_Maradona,84598,2400,82198
5,11,Alžběta_II.,110020,32368,77652
6,11,Viktorie_(britská_královna),79175,8889,70286
7,11,Hlavní_strana,1632511,1567999,64512
8,11,Seznam_prezidentů_Spojených_států_amerických,69344,9344,60000
9,11,Spojené_státy_americké,87987,30237,57750


### December

In [18]:
df = calculate_pageviews_difference_month(month=12)
df.to_csv('data/cswiki_pageviews_difference_2019_2020_December.tsv', sep='\t', index=False)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


,month,page_title,views_2020,views_2019,views_difference
0,12,Český_státní_svátek,127115,8275,118840
1,12,Jan_Mikolášek,54369,399,53970
2,12,Alžběta_II.,84662,34016,50646
3,12,App_Store,47804,439,47365
4,12,O_vánoční_hvězdě,42666,0,42666
5,12,Hlavní_strana,1451036,1409312,41724
6,12,"Margaret,_hraběnka_Snowdon",51404,9886,41518
7,12,Ladislav_Mrkvička,38494,1092,37402
8,12,Seznam_dílů_seriálu_Slunečná,36105,0,36105
9,12,Covid-19,35961,0,35961
